In [ ]:
!pip install langchain langchain-openai pymongo bs4 openai tiktoken gradio requests lxml argparse unstructured boto3

In [13]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import gradio as gr
from gradio.themes.base import Base
import key_param

In [2]:
# 各種変数の設定
client = MongoClient(key_param.MONGO_URI)
dbName = "langchain_demo_ja"
collectionName = "collection_of_text_blobs"
collection = client[dbName][collectionName]

In [3]:
# ディレクトリ内のテキストファイルをdataオブジェクトに読み込む
loader = DirectoryLoader( './sample_files_ja', glob="./*.txt", show_progress=True)
data = loader.load()

100%|███████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.16s/it]


In [4]:
# ベクトル化を行うためのモデルを定義
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', api_key=key_param.openai_api_key)

In [5]:
# data に対するベクトルを生成して、langchain_demo.collection_of_text_blobsコレクションに追加する
vectorStore = MongoDBAtlasVectorSearch.from_documents( data, embeddings, collection=collection, index_name='langchain_demo_ja' )

In [6]:
# 注：この先に進む前にAtlas UIでlangchain_demo.collection_of_text_blobsコレクションに対して
# 以下のVector Searchのインデックスを作成する

#index名: langchain_demo_ja

'''
{
  "fields": [{
    "path": "embedding",
    "numDimensions": 1536,
    "similarity": "cosine",
    "type": "vector"
  }]
}
'''

'\n{\n  "fields": [{\n    "path": "embedding",\n    "numDimensions": 1536,\n    "similarity": "cosine",\n    "type": "vector"\n  }]\n}\n'

In [22]:
def query_data(query):
    # OpenAIの埋め込みを使って質問をベクトルに変換し、
    # LangchainのvectorStoreを使って質問内容に関連のある文書をAtlas VectorSearchで検索し、
    # その文書の内容を考慮した回答を生成する

    # 質問文に最も関連がある文書1つをAtlas VectorSearchのsimilarity_searchで抽出
    docs = vectorStore.similarity_search(query, K=1)
    as_output = docs[0].page_content

    # 回答文生成に利用するOpenAIのLLMモデルを定義
    llm = OpenAI(model='gpt-3.5-turbo-instruct', api_key=key_param.openai_api_key, temperature=0)
    template = """Question: {question}
    Answer: """
    prompt = PromptTemplate.from_template(template)
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    raw_answer = llm_chain.run(query)

    # OpenAIの文書生成モデルとAtlas VectorSearchを用いて、質問文に関連がある文書を踏まえた回答を生成
    retriever = vectorStore.as_retriever()
    qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
    retriever_output = qa.invoke(query)

    # as_outout: 質問と関連が高い文書の内容
    # retriever_output: 上記の文書を踏まえて生成された回答内容
    return raw_answer, as_output, retriever_output['result']

In [23]:
# デモ画面の生成
with gr.Blocks(theme=Base(), title="Atlas Vector Search と RAG Architecture を使った質問回答アプリ") as demo:
    gr.Markdown(
        """
        #  Atlas Vector Search と RAG Architecture を使った質問回答アプリ
        質問例
        1. 2月13日の日経平均ニュースを教えてください
        2. アルフレッドはどのような質問をしましたか？
        3. アルフレッドはブルースとどのような会話をしましたか？CSATはどのくらいでしたか？
        4. yawとはなんですか?
        """)
    textbox = gr.Textbox(label="Enter your Question:")
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
    with gr.Column():
        output1 = gr.Textbox(lines=1, max_lines=10, label="Output with just LLM model:")
        output2 = gr.Textbox(lines=1, max_lines=10, label="Output with just Atlas Vector Search (returns text field as is):")
        output3 = gr.Textbox(lines=1, max_lines=10, label="Output generated by chaining Atlas Vector Search to Langchain's RetrieverQA + OpenAI LLM:")

# Call query_data function upon clicking the Submit button

    button.click(query_data, textbox, outputs=[output1, output2, output3])

demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [16]:
    # 回答文生成に利用するOpenAIのLLMモデルを定義
    question='2月13日のニュースを教えて'
    llm = OpenAI(model='gpt-3.5-turbo-instruct', api_key=key_param.openai_api_key, temperature=0)
    template = """Question: {question}
    Answer: """
    prompt = PromptTemplate.from_template(template)
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    result = llm_chain.run(question)

In [17]:
result

'2月13日のニュースは、以下のようなものがあります。\n\n1. 新型コロナウイルスの感染拡大が続く中、政府は緊急事態宣言を延長する方針を決定しました。延長期間は3月7日までとなります。\n\n2. 東京都では、新たに393人の新型コロナウイルス感染者が確認されました。また、重症者数も過去最多の142人となり、医療体制が逼迫しています。\n\n3. 日本政府は、新型コロナウイルスワクチンの接種を開始しました。まずは医療従事者を対象に、2月17日から接種が始まります。\n\n4. 北海道では、大雪による交通機関の乱れや停'

In [23]:
# 質問例
# 1. 2月１３日のニュースを教えてください
# 2. アルフレッドはどのような質問をしましたか？日本語の箇条書きでまとめてください。
# 3. アルフレッドはブルースとどのような会話をしましたか？CSATはどのくらいでしたか？
# 4. yawとはなんですか?

In [ ]:
# Question Answering App using Vector Search + RAG
# 1. Did any error occur on August 16th? If yes, then what was the error caused by?\n",
# 2. What questions did Alfread あsk?What were Bruce's answers? Please summarize in bullet points.\n",
# 3. What was the overall sentiment of Alfred's chat with Bruce? What was the likely CSAT?\n",
# 4. What's yaw?"